In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time
from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.neighbors import kneighbors_graph

In [45]:
def load_data(feature_file="features.csv", device="cuda"):
    df = pd.read_csv(feature_file)
    labels = torch.tensor(df["label"].values, dtype=torch.long, device=device)
    features = torch.tensor(df.drop(columns=["label"]).values, dtype=torch.float, device=device)
    return features, labels


In [46]:
def build_graph(features, threshold=0.9, device="cuda"):
    G = nx.Graph()
    num_nodes = features.shape[0]
    G.add_nodes_from(range(num_nodes))
    
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            similarity = torch.dot(features[i], features[j]) / (torch.norm(features[i]) * torch.norm(features[j]))
            if similarity > threshold:
                G.add_edge(i, j)
    
    edge_index = torch.tensor(list(G.edges), dtype=torch.long, device=device).t().contiguous()
    return G, edge_index

In [47]:
def build_graph(features, k=5, device="cuda"):
    adj_matrix = kneighbors_graph(features.cpu().numpy(), k, mode='connectivity', include_self=True)
    edge_index = np.array(adj_matrix.nonzero())
    edge_index = edge_index[:, edge_index[0] != edge_index[1]]  # Remove self-loops
    edge_index = torch.tensor(edge_index, dtype=torch.long, device=device)
    return edge_index.to(device).cpu().numpy().T  # Convert to list format for NetworkX

In [48]:
def visualize_graph(edge_index, num_nodes, device="cuda"):
    plt.figure(figsize=(8, 6))
    G = nx.Graph()
    edges = edge_index.tolist()
    G.add_edges_from(edges)
    G.add_nodes_from(range(num_nodes))
    pos = nx.spring_layout(G, seed=42)
    
    print("Rendering graph...")
    for i in tqdm(range(len(edges)), desc="Adding edges"):
        time.sleep(0.0001)  # Simulating real-time visualization update
        G.add_edge(*edges[i])
    
    nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray')
    plt.title("Graph Visualization")
    plt.show()

In [49]:
class GCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, out_dim)
    
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load data
features, labels = load_data(device=device)

In [50]:
edge_index = build_graph(features, device=device)

In [51]:
num_classes = len(torch.unique(labels))

In [ ]:
visualize_graph(edge_index, features.shape[0], device=device)

In [ ]:
tensor_edge_index = torch.tensor(edge_index.T, dtype=torch.long, device=device)

In [ ]:
data = Data(x=features, edge_index=tensor_edge_index, y=labels).to(device)

model = GCN(features.shape[1], 16, num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.NLLLoss()

# Training loop
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out, data.y)
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

print("Training complete.")
